In [2]:
import pandas as pd
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [3]:
train = np.asarray(pd.read_csv('train2.csv'))
dx_train , dy_train = train[:, :-1], train[:,-1]
x_test = np.asarray(pd.read_csv('test2.csv'))

In [4]:
p = np.random.permutation(dx_train.shape[0])
x_train = dx_train[p[0:int(len(dx_train)*0.8)], :]
y_train = dy_train[p[0:int(len(dx_train)*0.8)]]
x_val = dx_train[p[int(len(dx_train)*0.8):], :]
y_val = dy_train[p[int(len(dx_train)*0.8):]]

In [5]:
def preproc(x):
    colm_val_dict = {}
    for i in range(0,x.shape[1]):
        tmp = x[:, i]
        if type(tmp[0]) is str:
            count = 0
            value_dict = {}
            #составляем словарь
            for val in tmp:
                if not (val in value_dict.keys()):
                    if val == ' ?':
                        value_dict[val] = np.nan
                    else:
                        value_dict[val] = count
                        count+=1
            colm_val_dict[str(i)] = value_dict
            #print(value_dict)
            #заменяем
            for j in range(len(x[:, i])):
                x[:, i][j]= value_dict[x[:, i][j]]
    return x

In [6]:
def normolize(x):
    for i in range(0, x.shape[1]):
        tmp = x[:, i]
        x[:, i] = (x[:, i] - np.mean(x[:, i], axis = 0)) / np.var(x[:, i], axis = 0)
    return x

In [7]:
x_train = preproc(x_train)
x_test = preproc(x_test)
x_val = preproc(x_val)

In [8]:
imp_train = Imputer(missing_values=np.nan)
x_train = imp_train.fit_transform(x_train)

imp_val = Imputer(missing_values=np.nan)
x_val = imp_val.fit_transform(x_val)

imp_test = Imputer(missing_values=np.nan)
x_test = imp_test.fit_transform(x_test)

In [9]:
x_train = normolize(x_train)
x_val = normolize(x_val)
x_test = normolize(x_test)

In [10]:
x_train = [list(x) for x in x_train]
x_test = [list(x) for x in x_test]
x_val = [list(x) for x in x_val]
y_train = list(y_train)
y_val = list(y_val)

In [21]:
#KNN
k_range = list(range(10, 20))
param_grid_knn = dict(n_neighbors=k_range)
#SVM linear
param_grid_svc_lin = {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
#RBF SVM
param_grid_svc_rbf = {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]}
#Decision Tree
param_grid_dec_tree = {'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4]}
#Random Forest
param_grid_random_forest = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
#Neural Net
param_grid_mlp={
'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(100,1), (100,2), (100,3)],
'alpha': [10.0 ** -np.arange(1, 7)],
'activation': ["logistic", "relu", "Tanh"]
}
#AdaBoost
param_grid_adaboost = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "n_estimators": [1, 2]
             }

In [23]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM",
         "Decision Tree", "Random Forest"]
classifiers = [
    GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=10, scoring='accuracy', verbose=10),
    GridSearchCV(SVC(), param_grid_svc_lin,scoring='accuracy', verbose=10),
    GridSearchCV(SVC(), param_grid_svc_rbf,scoring='accuracy', verbose=10),
    GridSearchCV(DecisionTreeClassifier(), param_grid_dec_tree, scoring='accuracy', verbose=10),
    GridSearchCV(RandomForestClassifier(),param_grid_random_forest, scoring='accuracy', verbose=10)]

from sklearn.externals import joblib
 # iterate over classifiers
for name, clf in zip(names, classifiers):
    clf.fit(x_train, y_train)
    score = clf.score(x_val, y_val)
    print(name+": ", score)
    y_pred = clf.predict(x_test)
    f = open(f'lab2_Kozyrevskii5710M{name}-({score}).txt','w')
    for y in y_pred:
        f.write(str(y) +'\n')
    f.close()
    joblib.dump(clf.best_estimator_, f'{name}.pkl')

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] n_neighbors=10 ..................................................
[CV] ......... n_neighbors=10, score=0.8268714011516315, total=   0.6s
[CV] n_neighbors=10 ..................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s


[CV] ......... n_neighbors=10, score=0.8318618042226488, total=   0.6s
[CV] n_neighbors=10 ..................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.9s remaining:    0.0s


[CV] ......... n_neighbors=10, score=0.8257197696737044, total=   0.6s
[CV] n_neighbors=10 ..................................................


...

[CV]  bootstrap=False, criterion=entropy, max_depth=None, max_features=1, min_samples_leaf=10, min_samples_split=3, score=0.855448053443907, total=   0.0s
[CV] bootstrap=False, criterion=entropy, max_depth=None, max_features=1, min_samples_leaf=10, min_samples_split=10 
[CV]  bootstrap=False, criterion=entropy, max_depth=None, max_features=1, min_samples_leaf=10, min_samples_split=10, score=0.8583438903604745, total=   0.0s
[CV] bootstrap=False, criterion=entropy, max_depth=None, max_features=1, min_samples_leaf=10, min_samples_split=10 
[CV]  bootstrap=False, criterion=entropy, max_depth=None, max_features=1, min_samples_leaf=10, min_samples_split=10, score=0.8565012092594725, total=   0.0s
[CV] bootstrap=False, criterion=entropy, max_depth=None, max_features=1, min_samples_leaf=10, min_samples_split=10 
[CV]  bootstrap=False, criterion=entropy, max_depth=None, max_features=1, min_samples_leaf=10, min_samples_split=10, score=0.8498041925823543, total=   0.0s
[CV] bootstrap=False, crit

[CV]  bootstrap=False, criterion=entropy, max_depth=None, max_features=10, min_samples_leaf=1, min_samples_split=2, score=0.8408384199009559, total=   0.5s
[CV] bootstrap=False, criterion=entropy, max_depth=None, max_features=10, min_samples_leaf=1, min_samples_split=2 
[CV]  bootstrap=False, criterion=entropy, max_depth=None, max_features=10, min_samples_leaf=1, min_samples_split=2, score=0.8374985604053898, total=   0.5s
[CV] bootstrap=False, criterion=entropy, max_depth=None, max_features=10, min_samples_leaf=1, min_samples_split=2 
[CV]  bootstrap=False, criterion=entropy, max_depth=None, max_features=10, min_samples_leaf=1, min_samples_split=2, score=0.8412808108730707, total=   0.5s
[CV] bootstrap=False, criterion=entropy, max_depth=None, max_features=10, min_samples_leaf=1, min_samples_split=3 
[CV]  bootstrap=False, criterion=entropy, max_depth=None, max_features=10, min_samples_leaf=1, min_samples_split=3, score=0.8418749280202695, total=   0.4s
[CV] bootstrap=False, criterion

[Parallel(n_jobs=1)]: Done 648 out of 648 | elapsed:  2.2min finished


Random Forest:  0.7857800982800983
Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] activation=logistic, alpha=[1.e-01 1.e-02 1.e-03 1.e-04 1.e-05 1.e-06], hidden_layer_sizes=(100, 1), learning_rate=constant 


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()